## Creating flat ICE reference data suitable for point-in-time joining on CUSIP

Intercontinental Exchange (ICE) provides us with reference data for each instrument in an XML file.  This is uploaded to BigQuery with a Cloud Function: https://console.cloud.google.com/functions/details/us-east1/ice_ftp_to_gcs?project=eng-reactor-287421&tab=source

The following notebook flattens the nested ICE file and extracts features using views. The view _ice_flat_ is then combined with trade data in https://github.com/Ficc-ai/ficc/blob/jesse/SQL_examples/Create_trade_history_with_reference_data.ipynb

More detailed documentation and research regarding these fields is found in the ICE reference Wiki here: 
https://www.notion.so/ICE-Reference-Wiki-8d1a451a1233437eb369ffb3a6410d6e. 

NB: Eventually this notebook, like all notebooks, should have a requirements file and a comment should say where that is stored.

First we setup access to Big Query.

In [ ]:
import os
import datetime
from google.cloud import bigquery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/user/base/ficc/creds.json"
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/jupyter/creds.json"
bq_client = bigquery.Client()

project = "eng-reactor-287421"

Next we define a function that creates views. Conceptually, a view is a table that is defined by a query. When the view is created, the query is not evaluated. Instead, the query is evaluated whenever the table is used in a later query. The database system combines the query that defines the table with the later query, and optimizes them jointly.

The final view (_ice_flat_) is stored in a dataset called _primary_views_. This is the view that should be used by our models. Views that are made to create this view are stored in a dataset called _auxiliary_views_ to reduce clutter. These should not be used except to create _primary_views_.

In [ ]:
from datetime import date
def mkview(dataset,name,sql):
    db = f"{project}.{dataset}."
    name = db + name
    bq_client.delete_table(name, not_found_ok=True) 
    view = bigquery.Table(name)
    view.view_query = sql
    view = bq_client.create_table(view)
    return name

We will use the following function to test views. If the limit argument is missing, then the whole table is returned.

In [ ]:
def sqltodf(sql,limit = ""):
    if limit != "": 
        limit = f" ORDER BY RAND() LIMIT {limit}"
    bqr = bq_client.query(sql + limit).result()
    return bqr.to_dataframe()

In [ ]:
no_duplicates = mkview ("auxiliary_views","no_duplicates", 
f'''  
 select * from ( SELECT
    ROW_NUMBER() OVER (PARTITION BY CONCAT (instrument.id, ice_file_date)) as num_records,
    *
  FROM
    `eng-reactor-287421.reference_data.ice_nested`) where num_records < 2 
''')
print(no_duplicates)
%time df = sqltodf(f"SELECT * FROM {no_duplicates}", 3)
df

In order to get point-in-time reference data valid as of a given trade date, we create the following view with the calculated fields "_ref_valid_from_date_" and "_ref_valid_to_date_." We can then join MSRB transactions on two conditions (_MSRB.CUSIP_ = _ICE.CUSIP_) and (_MSRB.Trade_Date_ between _ICE.ref_valid_from_date_ and _ICE.ref_valid_to_date_). The valid_to date is one second before the file_date of the new record because BigQuerys' BETWEEN operator is inclusive.

More specific logic for this view: _ORDER_ASC_ and _ORDER_DESC_ show the order of the record among all records we have received for instrument.id.  For _ref_valid_from_date_ we use 2010-01-01 if this is the first record for any particular_instrument.id_; otherwise we use _ice_file_date_.  For _ref_valid_to_date_, if it is the last record the date will be '2100-01-01' to show that the record is valid for the foreseeable future (that is, until we receive another record for the same instrument.id.) Having a valid from date in the relatively distant past allows us to use ice_reference_date for trades that occurred prior to 2020-12-21.

In [ ]:
valid_time = mkview ("auxiliary_views","valid_time", 
f'''  
  SELECT
    ROW_NUMBER() OVER (PARTITION BY instrument.id ORDER BY ice_file_date DESC) ICE_RECORD_BY_ORDR_DESC,
    CASE
      WHEN ROW_NUMBER() OVER (PARTITION BY instrument.id ORDER BY ice_file_date ASC) = 1 THEN '2010-01-01'
    ELSE
    ice_file_date
  END
    ref_valid_from_date,
    CASE
      WHEN ROW_NUMBER() OVER (PARTITION BY instrument.id ORDER BY ice_file_date DESC) = 1 THEN '2100-01-01'
    ELSE
    TIMESTAMP_SUB (LEAD(ice_file_date) OVER (PARTITION BY instrument.id ORDER BY ice_file_date), INTERVAL 1 second)
  END
    ref_valid_to_date,
    CONCAT (instrument.id, ice_file_date) as key,
    *
  FROM
    {no_duplicates}
''')
print(valid_time)
%time df = sqltodf(f"SELECT * FROM {valid_time}", 3)
df

The following view associates ICE's instrument.id with a CUSIP, which enables us to join ICE data with other types of data like MSRB and historical S&P ratings. CUSIP is a value within the nested field _xref_ called "_xref.text_".  

The following auxiliary views use the pair _instrument_id_ and _ice_file_date_ as a key which enables us to create the primary view _ice_flat_.

In [ ]:
unnest_cusip = mkview ("auxiliary_views","unnest_cusip", 
f''' 
    SELECT
      key,
      xref.text AS cusip
    FROM
      {valid_time}
    LEFT JOIN
      UNNEST(instrument.master_information.instrument_xref.xref) AS xref
    ON
      xref.type='CUSIP'
    GROUP BY
      xref.text,
      key
''')
print(unnest_cusip)
%time df = sqltodf(f"SELECT * FROM {unnest_cusip}", 3)
df

The following two views create fields related to the callability of a bond. For more information on callability, please refer to the wiki page on the subject: https://www.notion.so/Callability-of-a-Muni-Bond-in-ICE-Reference-Data-51b8a3a4cec2483081877f5b76077be1

For an explation of _escrowed_or_pre_refunded_ see here: https://www.notion.so/Understanding-called_redemption_type-ef68f8218d5d4795a514239c70c5c5f1

"Refund_date" is a value that ICE supplies for the date at which a bond will be refunded.  It has been our practice to treat bonds that are escrowed to maturity as "called", but ICE does not supply a refund_date for bonds that are escrowed to maturity.  To prevent unnecessary complication downstream, we change refund_date to have the value for maturity_date in the case that the bond has been escrowed to maturity. 

Note: We use the IF condition here to avoid nulls. 

In [ ]:
callability = mkview ("auxiliary_views","callability",
f'''
    SELECT
        key,
    IF
    ( (instrument.debt.muni_details.called_redemption_type IN ( 1,
          5)AND (instrument.debt.muni_details.call_defeased <>"0"
          OR instrument.debt.muni_details.call_defeased IS NULL))
      OR ((instrument.debt.muni_details.called_redemption_type IN ( 13,
            18)
          AND instrument.debt.muni_details.refund_date IS NOT NULL))
      OR ((instrument.debt.muni_details.called_redemption_type IN ( 2,
            3,
            4,
            6,
            14,
            15,
            17)) ), TRUE, FALSE) AS is_called,
IF
  (instrument.debt.call_details.next_call_date IS NOT NULL
    AND (((instrument.debt.muni_details.called_redemption_type IS NULL
          OR instrument.debt.muni_details.called_redemption_type NOT IN ( 2,
            3,
            4,
            6,
            13,
            14,
            15,
            17))
        AND (instrument.debt.muni_details.called_redemption_type IS NULL
          OR instrument.debt.muni_details.called_redemption_type NOT IN (1,
            5)
          OR instrument.debt.muni_details.call_defeased = "0") ) ), TRUE, FALSE) AS is_callable,
  IF
    ((instrument.debt.muni_details.called_redemption_type IS NOT NULL
        OR instrument.debt.muni_details.called_redemption_type NOT IN (1,
          2,
          3,
          5,
          6,
          13,
          17)), TRUE, FALSE) AS is_escrowed_or_pre_refunded,
    CASE
      WHEN (instrument.debt.muni_details.called_redemption_type IN ( 1, 5)AND (instrument.debt.muni_details.call_defeased <>"0" OR instrument.debt.muni_details.call_defeased IS NULL)) THEN instrument.debt.fixed_income.maturity_date
    ELSE
    instrument.debt.muni_details.refund_date
     END
    refund_date, 
    case 
    WHEN 
    (instrument.debt.muni_details.called_redemption_type IN ( 1, 5)AND (instrument.debt.muni_details.call_defeased <>"0" OR instrument.debt.muni_details.call_defeased IS NULL)) THEN 100 
    ELSE
     instrument.debt.muni_details.refund_price
  END
    refund_price
    FROM {valid_time}
''')
print (callability)
%time df = sqltodf(f"select * from {callability}",3)
df

In [ ]:
call_details = mkview ("auxiliary_views","call_details",
f'''
    SELECT
      c.key,
    v.instrument.debt.call_details.call_schedule[SAFE_OFFSET (0)].call_date 
      AS first_call_date,
    v.instrument.debt.call_details.call_notice 
      AS call_date_notice,
    IF
      (v.instrument.debt.call_details.call_cav IS NOT NULL
        AND (v.instrument.debt.call_details.call_cav IS TRUE),
        TRUE,
        FALSE) AS callable_at_cav,
        v.instrument.debt.call_details.par_call_price AS par_price
    FROM {valid_time} as v
    LEFT JOIN {callability} as c
    ON v.key = c.key
''')
print (call_details)
%time df = sqltodf(f"select * from {call_details}",3)
df

The following view creates a Boolean "_default_exists_" for model training. For more detail on Defaults, please refer to the wiki page on the subject: https://www.notion.so/Defaults-430be93dfe8c4dbcadaebb641ebc1c1e

In [ ]:
lead_manager = mkview ("auxiliary_views","lead_manager", 
f''' 
    SELECT
      a.key,
      b.text,
      b.type,
      b.id as lead_manager_id,
      b.organization as lead_manager_org
    FROM
     {valid_time} a,
       UNNEST (instrument.global_information.relations.instrument_roles.role)b
    where b.text = "3"
    GROUP BY
    a.key,
    b.text,
    b.type,
    b.id,
    b.organization
''')
print(lead_manager)
%time df = sqltodf(f"select * from {lead_manager}",3)
df

In [ ]:
bond_counsel = mkview ("auxiliary_views","bond_counsel", 
f''' 
    SELECT
      a.key,
      b.text,
      b.type,
      b.id as bond_counsel_id,
      b.organization as bond_counsel_org
    FROM
     {valid_time} a,
       UNNEST (instrument.global_information.relations.instrument_roles.role)b
    where b.text = "12"
    GROUP BY
    a.key,
    b.text,
    b.type,
    b.id,
    b.organization
''')
print(bond_counsel)
%time df = sqltodf(f"select * from {bond_counsel}",3)
df

In [ ]:
financial_advisor = mkview ("auxiliary_views","financial_advisor", 
f''' 
    SELECT
      a.key,
      b.text,
      b.type,
      b.id as financial_advisor_id,
      b.organization as financial_advisor_org
    FROM
     {valid_time} a,
       UNNEST (instrument.global_information.relations.instrument_roles.role)b
    where b.text = "2"
''')
print(financial_advisor)
%time df = sqltodf(f"select * from {financial_advisor}",3)
df

In [ ]:
unnest_defaults = mkview ("auxiliary_views","unnest_defaults", 
f''' 
  SELECT
    key,
    count (de.default_date) > 0 AS default_exists
  FROM
    {valid_time}
  LEFT JOIN
    UNNEST(instrument.debt.default_event) AS de
  GROUP BY
    key
''')
print(unnest_defaults)
%time df = sqltodf(f"select * from {unnest_defaults}",3)
df

The following creates a Boolean for one type of bond with sinking funds: those paired with a mandatory par call option.  We create a Boolean as this type of sinking fund is likely to have an effect on YTW calculations.   For further reference on sink provisions, please refer to the wiki page on the subject here: https://www.notion.so/Sink-provisions-ed2e8501d1194d89b000e3893bf7d011

In [ ]:
unnest_sink_schedule = mkview("auxiliary_views","unnest_sink_schedule", 
f'''
  SELECT
    key,
    COUNT(sink_schedule) > 0 AS has_sink_schedule
  FROM
    {valid_time} 
  LEFT JOIN
    UNNEST (instrument.debt.sink_schedule) AS sink_schedule
  GROUP BY
    key
''')
print (unnest_sink_schedule)
%time df = sqltodf (f"select * from {unnest_sink_schedule}",3)
df

The following view creates a Boolean for unexpired lines of credit and a duration for time until expiration for lines of credit, should they exist.

In [ ]:
unnest_lines_of_credit = mkview ("auxiliary_views","unnest_lines_of_credit", 
f''' 
    SELECT
     key,
    IF
      ((max (loc_details.loc_expiration_date) > current_date),
        TRUE,
        FALSE) AS has_unexpired_lines_of_credit,
      date_diff (MAX(loc_details.loc_expiration_date), CAST(
        ice_file_date as DATE),
        year) AS years_to_loc_expiration
    FROM
     {valid_time}
    LEFT JOIN
      UNNEST (instrument.debt.loc_details) AS loc_details
    GROUP BY
      key,
      ice_file_date
''')
print (unnest_lines_of_credit)
%time df = sqltodf (f"select * from {unnest_lines_of_credit}", 3)
df

The following view creates a Boolean for instruments with a child issuance. 

In [ ]:
unnest_child_linkage = mkview ("auxiliary_views","unnest_child_linkage", 
f''' 
    SELECT
      key,
      count (unnest_child_linkage.instrument_id)> 0 AS child_linkage_exists
    FROM
      {valid_time}
    LEFT JOIN
      UNNEST(instrument.debt.muni_details.linkage_child.linkage) AS unnest_child_linkage
    GROUP BY
      key
''')
print (unnest_child_linkage)
%time df = sqltodf(f"select * from {unnest_child_linkage}",3)
df

The following view creates a Boolean for instruments secured by an escrow fund.  Escrow funds have been repeatedly used for corruption, see Mysak, *Encyclopedia* sub. "yield-burning" and "escrow churning."

In [ ]:
unnest_escrow = mkview ("auxiliary_views","unnest_escrow",
f'''
  SELECT
    key,
    count (muni_escrow.escrow_obligation_type)> 0 AS escrow_exists,
    MAX (muni_escrow.escrow_obligation_percent) AS escrow_obligation_percent,
    MAX( muni_escrow.escrow_obligation_agent) AS escrow_obligation_agent,
    MAX (muni_escrow.escrow_obligation_type) AS escrow_obligation_type
  FROM
    {valid_time}
  LEFT JOIN
    UNNEST (instrument.debt.muni_escrow) AS muni_escrow
  GROUP BY
    key
''')
print (unnest_escrow)
%time df = sqltodf (f"select * from {unnest_escrow}", 3)
df

The following view creates a fields for maximum amount outstanding and minimum amount outstanding. 

In [ ]:
unnest_amount_outstanding = mkview ("auxiliary_views","unnest_amount_outstanding",
f'''
  SELECT
    key,
    Max (history_change.amount_outstanding) AS max_amount_outstanding,
    min (history_change.amount_outstanding) AS min_amount_outstanding
  FROM
    {valid_time}
  LEFT JOIN
    UNNEST (instrument.debt.amount_outstanding_history.change) AS history_change
  GROUP BY
    key
''')
print (unnest_amount_outstanding)
%time df = sqltodf (f"select * from {unnest_amount_outstanding}",3)
df

The following view creates a Boolean to determine whether an event exists.  There are 70 types of event defined by ICE  based on SEC reporting requirements.  A next step to create booleans for each of the 70 types of event, to think about how to tie events to timestamps, and to identify which of types of event are more significant. For more details on events see the relevant wiki page: https://www.notion.so/Events-de11fdbc907b4cc79d84a047aea4e02e

In [ ]:
unnest_event = mkview ("auxiliary_views","unnest_event",
f'''
  SELECT
    key, 
    COUNT(event.code)> 0 AS event_exists
  FROM
    {valid_time}
  LEFT JOIN
    UNNEST (instrument.debt.event) AS event
  GROUP BY
    key
''')
print (unnest_event)
%time df = sqltodf (f"select * from {unnest_event}", 3)
df

The following creates a view for ratings in the ICE data. 

ICE's S&P ratings are valid point-in-time because they come from the ICE data. Ratings obtained from S&P or elsewhere would need to be associated with dates, in order to be valid point-in-time. Joining them would require non-trivial, but doable, logic.

In [ ]:
unnest_ratings = mkview ("auxiliary_views","unnest_ratings", 
f'''
  SELECT
    key,
    MAX(IF(rating.type="Long Rating" and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_long,
    MAX(IF(rating.type="Issue Long Rating"and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_stand_alone,
    MAX(IF(rating.type="Preliminary Long Rating"and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_prelim_long,
    MAX(IF(rating.type="School ICR Long Rating"and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_icr_school, 
    MAX(IF(rating.type="Credit Watch Long Outlook Rating"and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_outlook_long,
    MAX(IF(rating.type="Short Rating"and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_Short_Rating,
    MAX(IF(rating.type="Credit Watch Short Rating"and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_Credit_Watch_Short_Rating,
    MAX(IF(rating.type="Recovery Long Rating"and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_Recovery_Long_Rating,
    MAX(IF(rating.type="Credit Watch Long Rating"and rating.agency = "Standard & Poor's",rating.text,NULL)) AS sp_watch_long,
    MAX(IF(rating.type="Long Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_long,
    MAX(IF(rating.type="Short Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_short,
    MAX(IF(rating.type="Issue Long Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_Issue_Long_Rating,
    MAX(IF(rating.type="Issue Short Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_Issue_Short_Rating,
    MAX(IF(rating.type="Credit Watch Long Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_Credit_Watch_Long_Rating,
    MAX(IF(rating.type="Credit Watch Short Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_Credit_Watch_Short_Rating,
    MAX(IF(rating.type="Enhanced Long Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_Enhanced_Long_Rating,
    MAX(IF(rating.type="Enhanced Short Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_Enhanced_Short_Rating,
    MAX(IF(rating.type="Credit Watch Long Outlook Rating" and rating.agency = "Moody's",rating.text,NULL)) AS moodys_Credit_Watch_Long_Outlook_Rating,
  FROM
    {valid_time}
  LEFT JOIN
    UNNEST(instrument.global_information.ratings.rating) AS rating
  GROUP BY
    key
''')
print (unnest_ratings)
%time df = sqltodf (f"select * from {unnest_ratings}",3)
df

The following view creates boolean fields for various characteristics of a bond not included in the above views. 

A wiki for General Obligation (GO) is here:
https://www.notion.so/ICE-Reference-Wiki-8d1a451a1233437eb369ffb3a6410d6e?p=e1641b3220fe41fe9e51392e581dc153&showMoveTo=true

In [ ]:
booleans = mkview ("auxiliary_views","booleans", 
f'''
    SELECT
      key,
    IF
        ((instrument.debt.fixed_income.bond_insurance IS NOT NULL
        AND instrument.debt.fixed_income.bond_insurance <> "NONE"),
        TRUE,
        FALSE) AS bond_insurance_exists,
    IF
      (instrument.debt.floating_rate_instruments.benchmark_code IS NULL,
        FALSE,
        TRUE) AS floating_rate_exists,
    IF
      ( (instrument.debt.put_details.put_type <> 16 and instrument.debt.put_details.put_type is not null),
        true,
        false) AS put_exists,
    IF
      (instrument.debt.muni_details.muni_security_type=6
        OR instrument.debt.muni_details.muni_security_type = 5,
        TRUE,
        FALSE) AS is_general_obligation,
    IF 
       (instrument.debt.fixed_income.coupon_type=4,
       TRUE,
       FALSE) as has_zero_coupons
    FROM
        {valid_time}
''')
print (booleans)
%time df = sqltodf (f"select * from {booleans}",3)
df       

The following view creates the primary _ice_flat_ view.

In [ ]:
ice_flat_before_ratings_change = mkview ("auxiliary_views","ice_flat_before_ratings_change",
f'''
    SELECT
      ice_file_date,
      instrument.id,
      unnest_cusip.cusip,
      ref_valid_from_date,
      ref_valid_to_date,
      instrument.master_information.organization_master.incorporated_state_code,
      instrument.master_information.organization_master.primary_name AS organization_primary_name,
      instrument.master_information.organization_master.id AS ice_organization_id,
      instrument.master_information.instrument_master.primary_name AS instrument_primary_name,
      instrument.debt.muni_details.issue_key,
      instrument.debt.muni_details.issue_text,
      instrument.debt.muni_details.conduit_obligor_name,
      callability.is_called, 
      callability.is_callable,
      callability.is_escrowed_or_pre_refunded,
      callability.refund_date,
      callability.refund_price,
      call_details.first_call_date,
      call_details.call_date_notice,
      call_details.callable_at_cav,
      call_details.par_price,
      --bond_counsel.bond_counsel_id,
      --bond_counsel.bond_counsel_org,
      --financial_advisor.financial_advisor_id,
      --financial_advisor.financial_advisor_org,
      --lead_manager.lead_manager_id,
      --lead_manager.lead_manager_org,
      instrument.debt.muni_details.call_defeased,
      instrument.debt.call_details.call_timing,
      instrument.debt.call_details.call_timing_in_part,
      instrument.debt.call_details.extraordinary_make_whole_call,
      instrument.debt.call_details.extraordinary_redemption,
      instrument.debt.call_details.make_whole_call,
      instrument.debt.call_details.next_call_date,
      instrument.debt.call_details.next_call_price,
      instrument.debt.call_details.num as call_redemption_id,
      instrument.debt.call_details.optional_redemption_code[safe_offset(0)] as first_optional_redemption_code,
      instrument.debt.call_details.optional_redemption_code[safe_offset(1)] as second_optional_redemption_code,
      instrument.debt.call_details.optional_redemption_code[safe_offset(2)] as third_optional_redemption_code,
      instrument.debt.call_details.mandatory_redemption_code[safe_offset(0)] as first_mandatory_redemption_code,
      instrument.debt.call_details.mandatory_redemption_code[safe_offset(1)] as second_mandatory_redemption_code,
      instrument.debt.call_details.mandatory_redemption_code[safe_offset(2)] as third_mandatory_redemption_code,
      instrument.debt.call_details.par_call_date,
      instrument.debt.call_details.par_call_price,
      instrument.debt.call_details.maximum_call_notice_period,
      instrument.debt.muni_details.called_redemption_type,
      instrument.debt.muni_details.muni_issue_type,
      --instrument.debt.muni_details.refund_date, REPLACED WITH CALLABILITY REFUND DATE FOR ETMs, SEE ABOVE
      --instrument.debt.muni_details.refund_price, as above
      instrument.debt.muni_details.redemption_cav_flag,
      instrument.debt.put_details.max_notification_days,
      instrument.debt.put_details.min_notification_days,
      instrument.debt.put_details.next_put_date,
      instrument.debt.put_details.put_end_date,
      instrument.debt.put_details.put_feature_price,
      instrument.debt.put_details.put_frequency,
      instrument.debt.put_details.put_start_date,
      instrument.debt.put_details.put_type,
      instrument.debt.fixed_income.maturity_date,
      unnest_ratings.sp_long, 
      unnest_ratings.sp_stand_alone, 
      unnest_ratings.sp_icr_school, 
      unnest_ratings.sp_prelim_long, 
      unnest_ratings.sp_outlook_long, 
      unnest_ratings.sp_watch_long,
      unnest_ratings.sp_Short_Rating,
      unnest_ratings.sp_Credit_Watch_Short_Rating,
      unnest_ratings.sp_Recovery_Long_Rating,
      unnest_ratings.moodys_long,
      unnest_ratings.moodys_short,
      unnest_ratings.moodys_Issue_Long_Rating,
      unnest_ratings.moodys_Issue_Short_Rating,
      unnest_ratings.moodys_Credit_Watch_Long_Rating,
      unnest_ratings.moodys_Credit_Watch_Short_Rating,
      unnest_ratings.moodys_Enhanced_Long_Rating,
      unnest_ratings.moodys_Enhanced_Short_Rating,
      unnest_ratings.moodys_Credit_Watch_Long_Outlook_Rating,
      unnest_sink_schedule.has_sink_schedule,
      instrument.debt.sink_details.next_sink_date,
      --instrument.debt.sink_details.sink_fund_type,
      instrument.debt.fixed_income.sink_indicator,
      instrument.debt.sink_details.mandatory_sink_amount_type.text as sink_amount_type_text,
      instrument.debt.sink_details.mandatory_sink_amount_type.type as sink_amount_type_type,
      instrument.debt.sink_details.sink_frequency,
      instrument.debt.muni_details.sink_defeased,
      instrument.debt.additional_sink_details.next_sink_date AS additional_next_sink_date,
      instrument.debt.additional_sink_details.sink_amount_type,
      instrument.debt.additional_sink_details.sink_frequency AS additional_sink_frequency, 
      unnest_amount_outstanding.min_amount_outstanding,
      unnest_amount_outstanding.max_amount_outstanding,
      unnest_defaults.default_exists,
      unnest_event.event_exists,
      unnest_lines_of_credit.has_unexpired_lines_of_credit,
      unnest_lines_of_credit.years_to_loc_expiration,
      unnest_escrow.escrow_exists,
      unnest_escrow.escrow_obligation_percent,
      unnest_escrow.escrow_obligation_agent,
      unnest_escrow.escrow_obligation_type,
      unnest_child_linkage.child_linkage_exists,
      booleans.put_exists, 
      booleans.floating_rate_exists, 
      booleans.bond_insurance_exists,
      booleans.is_general_obligation,
      booleans.has_zero_coupons,
      instrument.debt.floating_rate_instruments.next_reset_date,
      instrument.debt.floating_rate_instruments.first_variable_reset_date,     
      instrument.master_information.instrument_master.delivery_date,
      instrument.master_information.instrument_master.issue_price,
      instrument.master_information.instrument_master.settlement_date as primary_market_settlement_date,
      instrument.master_information.instrument_master.issue_date,
      instrument.master_information.instrument_master.outstanding_indicator,
      instrument.master_information.instrument_master.federal_tax_status,
      instrument.global_information.instrument_details.maturity_details.maturity_amount,
      instrument.global_information.instrument_details.denomination_amounts.available_denom,
      instrument.global_information.instrument_details.denomination_amounts.denom_increment_amount,
      instrument.global_information.instrument_details.denomination_amounts.min_denom_amount,
      instrument.debt.fixed_income.accrual_date,
      instrument.debt.fixed_income.bond_insurance,
      instrument.debt.fixed_income.coupon_type,
      instrument.debt.fixed_income.current_coupon_rate,
      instrument.debt.fixed_income.daycount_basis_type,
      instrument.debt.fixed_income.debt_type,
      instrument.debt.fixed_income.default_indicator,
      instrument.debt.fixed_income.first_coupon_date,
      instrument.debt.fixed_income.interest_payment_frequency,
      instrument.debt.fixed_income.issue_amount,
      instrument.debt.fixed_income.last_period_accrues_from_date,
      instrument.debt.fixed_income.next_coupon_payment_date,
      instrument.debt.fixed_income.odd_first_coupon_date,
      instrument.debt.fixed_income.other_accrual_date,
      instrument.debt.fixed_income.orig_principal_amount,
      instrument.debt.fixed_income.orig_avg_life_date,
      instrument.debt.fixed_income.original_yield,
      instrument.debt.fixed_income.outstanding_amount,
      instrument.debt.fixed_income.previous_coupon_payment_date,
      instrument.debt.fixed_income.sale_type,
      instrument.debt.fixed_income.settlement_type,
      instrument.debt.muni_details.additional_project_txt,
      instrument.debt.muni_details.asset_claim_code,
      instrument.debt.muni_details.additional_state_code,
      instrument.debt.muni_details.backed_underlying_security_id,
      instrument.debt.muni_details.bank_qualified,
      instrument.debt.muni_details.capital_type,
      instrument.debt.muni_details.conditional_call_date,
      instrument.debt.muni_details.conditional_call_price,
      instrument.debt.muni_details.designated_termination_date,
      instrument.debt.muni_details.DTCC_status,
      instrument.debt.muni_details.first_execution_date,
      instrument.debt.muni_details.formal_award_date,
      instrument.debt.muni_details.maturity_description_code,
      instrument.debt.muni_details.muni_security_type,
      instrument.debt.muni_details.mtg_insurance, 
      instrument.debt.muni_details.orig_cusip_status,
      instrument.debt.muni_details.orig_instrument_enhancement_type,
      instrument.debt.muni_details.other_enhancement_type,
      instrument.debt.muni_details.other_enhancement_company,
      instrument.debt.muni_details.pac_bond_indicator,
      instrument.debt.muni_details.project_name,
      instrument.debt.muni_details.purpose_class,
      instrument.debt.muni_details.purpose_sub_class,
      instrument.debt.muni_details.refunding_issue_key,
      instrument.debt.muni_details.refunding_dated_date,
      instrument.debt.muni_details.sale_date,
      instrument.debt.muni_details.sec_regulation,
      instrument.debt.muni_details.secured,
      instrument.debt.muni_details.series_name,
      instrument.debt.muni_details.sink_fund_redemption_method,
      instrument.debt.muni_details.state_tax_status,
      instrument.debt.muni_details.tax_credit_frequency,
      instrument.debt.muni_details.tax_credit_percent,
      instrument.debt.muni_details.use_of_proceeds,
      instrument.debt.muni_details.use_of_proceeds_supplementary,
      array_reverse (instrument.debt.event) as material_event_history,
      array_reverse (instrument.debt.default_event) as default_event_history,
       CASE
      WHEN unnest_ratings.moodys_long = 'Aaa' THEN 1
      WHEN unnest_ratings.moodys_long = 'Aa1' THEN 2
      WHEN unnest_ratings.moodys_long = 'Aa2' THEN 3
      WHEN unnest_ratings.moodys_long = 'Aa3' THEN 4
      WHEN unnest_ratings.moodys_long = 'A1' THEN 5
      WHEN unnest_ratings.moodys_long = 'A2' THEN 6
      WHEN unnest_ratings.moodys_long = 'A3' THEN 7
      WHEN unnest_ratings.moodys_long = 'Baa1' THEN 8
      WHEN unnest_ratings.moodys_long = 'Baa2' THEN 9
      WHEN unnest_ratings.moodys_long = 'Baa3' THEN 10
      WHEN unnest_ratings.moodys_long = 'Ba1' THEN 11
      WHEN unnest_ratings.moodys_long = 'Ba2' THEN 12
      WHEN unnest_ratings.moodys_long = 'Ba3' THEN 13
      WHEN unnest_ratings.moodys_long = 'B1' THEN 14
      WHEN unnest_ratings.moodys_long = 'B2' THEN 15
      WHEN unnest_ratings.moodys_long = 'B3' THEN 16
      WHEN unnest_ratings.moodys_long = 'Caa1' THEN 17
      WHEN unnest_ratings.moodys_long = 'Caa2' THEN 18
      WHEN unnest_ratings.moodys_long = 'Caa3' THEN 19
      WHEN unnest_ratings.moodys_long = 'Ca' THEN 20
      WHEN unnest_ratings.moodys_long = 'C' THEN 21
      ELSE 100 -- For any other value not matched, you can use -1 or any other default value
    END AS moodys_long_integer,
    CASE
      WHEN unnest_ratings.sp_long = 'AAA' THEN 1
      WHEN unnest_ratings.sp_long = 'AA+' THEN 2
      WHEN unnest_ratings.sp_long = 'AA' THEN 3
      WHEN unnest_ratings.sp_long = 'AA-' THEN 4
      WHEN unnest_ratings.sp_long = 'A+' THEN 5
      WHEN unnest_ratings.sp_long = 'A' THEN 6
      WHEN unnest_ratings.sp_long = 'A-' THEN 7
      WHEN unnest_ratings.sp_long = 'BBB+' THEN 8
      WHEN unnest_ratings.sp_long = 'BBB' THEN 9
      WHEN unnest_ratings.sp_long = 'BBB-' THEN 10
      WHEN unnest_ratings.sp_long = 'BB+' THEN 11
      WHEN unnest_ratings.sp_long = 'BB' THEN 12
      WHEN unnest_ratings.sp_long = 'BB-' THEN 13
      WHEN unnest_ratings.sp_long = 'B+' THEN 14
      WHEN unnest_ratings.sp_long = 'B' THEN 15
      WHEN unnest_ratings.sp_long = 'B-' THEN 16
      WHEN unnest_ratings.sp_long = 'CCC+' THEN 17
      WHEN unnest_ratings.sp_long = 'CCC' THEN 18
      WHEN unnest_ratings.sp_long = 'CCC-' THEN 19
      WHEN unnest_ratings.sp_long = 'CC' THEN 20
      WHEN unnest_ratings.sp_long = 'C' THEN 21
      WHEN unnest_ratings.sp_long = 'D' THEN 22
      ELSE 100 -- For any other value not matched, you can use -1 or any other default value
    END AS sp_long_integer,
    instrument.debt.partial_redemption
     FROM
      {valid_time} AS nested
    INNER JOIN
      {unnest_lines_of_credit} as unnest_lines_of_credit
    ON
      unnest_lines_of_credit.key = nested.key
    INNER JOIN
      {unnest_event} as unnest_event
    ON
      unnest_event.key = nested.key
    INNER JOIN
      {unnest_cusip} as unnest_cusip
    ON
      unnest_cusip.key = nested.key
    INNER JOIN
      {unnest_defaults} as unnest_defaults
    ON
      unnest_defaults.key = nested.key
    INNER JOIN
      {unnest_child_linkage} as unnest_child_linkage
    ON
      unnest_child_linkage.key = nested.key
    INNER JOIN
      {unnest_escrow} as unnest_escrow
    ON
      unnest_escrow.key = nested.key
    INNER JOIN
      {unnest_amount_outstanding} as unnest_amount_outstanding
    ON
      unnest_amount_outstanding.key = nested.key
    INNER JOIN
      {unnest_ratings} as unnest_ratings
    ON
      unnest_ratings.key = nested.key
    INNER JOIN 
        {unnest_sink_schedule} as unnest_sink_schedule
    ON
      unnest_sink_schedule.key = nested.key
    INNER JOIN 
        {callability} as callability
    ON
      callability.key = nested.key
    INNER JOIN 
        {call_details} as call_details
    ON 
      call_details.key = nested.key
    INNER JOIN
        {booleans} as booleans
    ON 
      booleans.key = nested.key
    --LEFT JOIN
    --    {financial_advisor} as financial_advisor 
    --ON 
    --  financial_advisor.key = nested.key
    --INNER JOIN
    --    {lead_manager} as lead_manager 
    --ON 
    --  lead_manager.key = nested.key
    --INNER JOIN
    --    {bond_counsel} as bond_counsel
    --ON 
    --  bond_counsel.key = nested.key
''')
print (ice_flat_before_ratings_change) 
%time df = sqltodf (f"select * from {ice_flat_before_ratings_change}",3)
df

In [ ]:
ice_flat = mkview ("primary_views","ice_flat",
f'''
  SELECT
  b.*,
    CASE
      WHEN b.other_enhancement_type IS NOT NULL AND (b.other_enhancement_type > 34 OR b.other_enhancement_type IN (20, 21, 28, 32)) THEN CAST(other_enhancement_type AS STRING)
      WHEN b.conduit_obligor_name IS NOT NULL THEN b.conduit_obligor_name
    ELSE
    b.organization_primary_name
  END
    AS obligor_id,
  CASE
    WHEN ((b.moodys_long_integer = a.min_moodys_rating AND a.min_moodys_rating <> a.max_moodys_rating AND a.min_moodys_rating < a.max_moodys_rating) OR (b.sp_long_integer = a.min_sp_rating AND a.min_sp_rating <> a.max_sp_rating AND a.min_sp_rating < a.max_sp_rating )) THEN TRUE
  ELSE
  FALSE
END
  AS rating_upgrade,
  CASE
    WHEN ((b.moodys_long_integer = a.max_moodys_rating AND a.min_moodys_rating <> a.max_moodys_rating AND a.max_moodys_rating > a.min_moodys_rating) OR (b.sp_long_integer = a.max_sp_rating AND a.min_sp_rating <> a.max_sp_rating AND a.max_sp_rating > a.min_sp_rating)) THEN TRUE
  ELSE
  FALSE
END
  AS rating_downgrade,
  a.min_sp_rating as min_sp_rating_this_year,
  a.max_sp_rating as max_sp_rating_this_year,
  a.min_moodys_rating as min_moodys_rating_this_year, 
  a.max_moodys_rating as max_moodys_rating_this_year,
 CASE
    WHEN ((b.moodys_long_integer = a.max_moodys_rating and b.moodys_long_integer <> 100 AND a.min_moodys_rating <> a.max_moodys_rating AND a.max_moodys_rating > a.min_moodys_rating and b.moodys_long_integer > 9 ) OR (b.sp_long_integer = a.max_sp_rating AND b.sp_long_integer <> 100 and a.min_sp_rating <> a.max_sp_rating AND a.max_sp_rating > a.min_sp_rating and b.sp_long_integer > 9)) THEN TRUE
  ELSE
  FALSE
END
  AS rating_downgrade_to_junk,
FROM
 eng-reactor-287421.auxiliary_views.ice_flat_before_ratings_change AS b
LEFT JOIN (
  SELECT
    *
  FROM (
    SELECT
      cusip,
      MAX(sp_long_integer) AS max_sp_rating,
      MIN(sp_long_integer) AS min_sp_rating,
      MAX(moodys_long_integer) AS max_moodys_rating,
      MIN(moodys_long_integer) AS min_moodys_rating,
    FROM
      eng-reactor-287421.auxiliary_views.ice_flat_before_ratings_change
    WHERE
      DATE_DIFF(current_date,DATE(ref_valid_from_date), month) < 12
    GROUP BY
      cusip ) ) AS a
ON
  a.cusip = b.cusip
''')
print (ice_flat)
%time df = sqltodf (f"select * from {ice_flat}", 3)
df

Finally we test whether any data have been lost by comparing the total rows in _ICE_flat_ with those in _ICE_nested_.

In [ ]:
query_ice_nested = ("SELECT instrument.id FROM `eng-reactor-287421.reference_data.ice_nested` where instrument.id is not null")
query_ice_flat = ("SELECT cusip FROM `eng-reactor-287421.primary_views.ice_flat` ")
query_flat_duplicates = ("select * from (select distinct concat(ref_valid_from_date, id), count(concat(ref_valid_from_date, id)) cnt from `primary_views.ice_flat` group by concat(ref_valid_from_date, id)) where cnt > 1")
query_nested_duplicates = ("select * from (select distinct concat(ice_file_date, instrument.id), count(concat(ice_file_date, instrument.id)) cnt from `reference_data.ice_nested` group by concat(ice_file_date, instrument.id)) where cnt > 1")


def count_total_rows (query):
    query_job = bq_client.query(
    query,
    location="US")
    results = query_job.result()
    return format(results.total_rows)

def check_row_totals(): 
    nested_total = int(count_total_rows (query_ice_nested))
    nested_duplicates = int(count_total_rows(query_nested_duplicates))
    flat_total = int(count_total_rows (query_ice_flat))
    flat_duplicates = int(count_total_rows(query_flat_duplicates))
    nested_without_duplicates = nested_total - nested_duplicates
    if flat_duplicates > 0:
        print(f"Warning!! There are {flat_duplicates} duplicates in ice_flat.")
    elif  nested_without_duplicates == flat_total:
        print(f"ICE_flat has the same number of rows as ICE_nested. ICE_nested has {nested_without_duplicates} rows (duplicates removed). ICE_flat has {flat_total} rows.")
    else: 
        print(f"ICE_flat does not have the same number of rows as ICE_nested. ICE_nested has {nested_without_duplicates} rows(duplicates removed). ICE_flat has {flat_total} rows.")

check_row_totals()

In [ ]:
df = sqltodf (f"select max(ref_valid_from_date) as most_recent_file_datetime from {ice_flat}",1)
most_recent_file_datetime = df["most_recent_file_datetime"][0].strftime("%m/%d/%Y, %H:%M:%S")
print(f'The most recent file datetime is {most_recent_file_datetime}')